In [1]:
import pandas as pd
import numpy as np
import multiprocessing as mp

from Levenshtein import ratio
from os import cpu_count
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer

In [2]:
# import nltk
# nltk.download('stopwords')

In [3]:
year = 2013
df = pd.read_parquet(f'../datasets/raw/{year}.parquet', engine='pyarrow')
df = df[['resumo', 'palavra_chave', 'subareas', 'areas', 'colegios']]

In [4]:
df.palavra_chave.replace(to_replace = r'1[.]', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' \d[.]', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'(\S{2,})(?:\.)', value = '\\1;', regex=True, inplace=True)
df.palavra_chave.replace(to_replace = r'\b(pt|en)\b', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' ,|, | [,] | /|/ | [/] ', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' [.] |\(\d\)', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' \x96 |\x93', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'[^a-z0-9\s\x80-\xff,.:;?!-]', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' - |; | ;|; ;', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r';{2,}', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace= r'[\W_]+$|^[;]+', value='', regex=True, inplace=True)
df.palavra_chave.replace(to_replace = r'palavras-chave\S', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'^\s+|\s+$|\x80', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'[^\w\s;]', value = '', regex = True, inplace=True)

In [5]:
df.resumo.replace(to_replace= r'[^a-zA-Z0-9\s]+', value='', regex=True, inplace=True)

In [6]:
def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
    return " ".join(new_text)

In [7]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
    return " ".join(words)

In [8]:
df.resumo = df.resumo.apply(remove_stopwords).apply(lc_stem).apply(lambda x: unidecode(x))
df.palavra_chave = df.palavra_chave.apply(remove_stopwords).apply(lc_stem).apply(lambda x: unidecode(x))
df.palavra_chave.replace(to_replace = r' ; ', value = ';', regex = True, inplace=True)
del remove_stopwords, lc_stem

In [9]:
df.drop(df.query("palavra_chave == '' | resumo == ''").index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
key_words = df.palavra_chave.copy()
key_words = list(set(';'.join(key_words).split(';')))

key_words = [item.replace(r'[^a-zA-Z0-9\s]+', '') for item in key_words]

key_words = [item for item in key_words if 2 < len(item) < 80]
key_words = [item for item in key_words if not item.isdigit()]
for i, _ in enumerate(key_words):
    key_words[i] = key_words[i].strip()

key_words = list(set(key_words))

len(key_words)

109190

In [11]:
# df_test_without_space = df.copy()
# df_test_without_space.palavra_chave.replace(r' ', '', regex=True, inplace=True)
# df_test_without_space.resumo.replace(r' ', '', regex=True, inplace=True)

# test_without_space = [item.replace(' ', '') for item in key_words]
# test_without_space = list(set(test_without_space))

# len(test_without_space)

In [36]:
def string_similarity(keyword: str, target: str) -> str:
    if keyword == '' or target == '': return False
    if keyword in target or target in keyword: return True
    
    similarity = ratio(keyword, target)*100
    
    if similarity > 75: return True
    
    return False

In [37]:
def apply_similarity(row, text):
    key = string_similarity(text, row['palavra_chave'])
    abstract = string_similarity(text, row['resumo'])
    return key or abstract

- 50 palavras_chave = 1m 1.3s
- 500 palavras_chave = 9 24.4s
- 5000 palavras_chave = 126m 2.3s
- 109190 palavras_chave = ?

In [ ]:
def process_character(keyword):    
    cut = df[df.apply(lambda row: apply_similarity(row, keyword), axis=1)]
    
    results = []
    for subarea in sorted(cut.subareas.unique()):
        new_cut = cut[cut.subareas == subarea]
        colegio, area = new_cut.colegios.unique()[0], new_cut.areas.unique()[0]
        frequencia = new_cut.palavra_chave.str.count(keyword).sum() + new_cut.resumo.str.count(keyword).sum()
        if not frequencia: frequencia = new_cut.shape[0]
        results.append([colegio, area, subarea, keyword, frequencia])
    
    return results

In [82]:
max_threads = cpu_count()
data = []
with mp.Pool(processes=max_threads) as pool:
    for result in pool.imap_unordered(process_character, key_words):
        data.extend(result)

In [83]:
columns_dtype = {
    'colegio': 'category',
    'area': 'category',
    'subarea': 'category',
    'palavra_chave': 'category',
    'frequencia': 'uint64'
}

freq = pd.DataFrame(data, columns=columns_dtype.keys()).astype(columns_dtype)

In [84]:
freq.set_index('palavra_chave').sort_index().sort_values('frequencia', ascending=False)

,colegio,area,subarea,frequencia
palavra_chave,,,,
ent,ciências da vida,ciências agrárias,ciências agrárias i,49386
ent,humanidades,ciências humanas,educação,42098
con,humanidades,ciências humanas,educação,35789
pro,humanidades,ciências humanas,educação,34852
ent,"ciências exatas, tecnológicas e multidisciplinar",multidisciplinar,interdisciplinar,33956
...,...,...,...,...
parafas,humanidades,ciências humanas,psicologia,1
parafas,ciências da vida,ciências da saúde,medicina ii,1
parafiletico,ciências da vida,ciências biológicas,biodiversidade,1


In [85]:
freq.to_parquet(f'../datasets/processed/frequency_matrix_{year}.parquet', engine='pyarrow')